<a href="https://colab.research.google.com/github/Rishika3D/Medical_Report_Validator_with_disease_prediction_using_blockchain_and_ML/blob/main/pancreas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 1. SETUP AND DEPENDENCIES
# ==============================================================================
import ast
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPool1D, Reshape, BatchNormalization, Activation, Add, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define paths and constants
PTB_XL_META_FILENAME = 'ptbxl_database (2).csv'
# The signal files (e.g., records500/00000/00001_hr.npy) are needed for high-accuracy models.
# They are not provided here and must be downloaded separately. The code assumes
# they are locally accessible in the Colab environment. The full dataset is large.
# For simplicity and a runnable example, we will proceed assuming the user
# has the full PTB-XL dataset (signal files) available or can manually download.
# A small sample (100 Hz, or 'lr') is used here for demonstration purposes only.
# For 95%+ accuracy, the full 500 Hz ('hr') or a powerful downloaded preprocessed version is needed.
# Since the full dataset download is complex, this code assumes a
# pre-downloaded 100Hz version, which is manageable to download from the repo.
DATA_PATH = '' # Assuming files are in the working directory
SAMPLE_RATE = 100 # Using 100 Hz ('lr') for faster prototyping. Change to 500 for 'hr' and higher accuracy.
N_SAMPLES = 1000 # 10 seconds of 100 Hz data
N_LEADS = 12
BATCH_SIZE = 64
EPOCHS = 40 # Set higher for final training (e.g., 100)

# ==============================================================================
# 2. DATA DOWNLOAD AND LOADING UTILITIES
# ==============================================================================

# Download the auxiliary files needed for mapping SC codes to superclasses
print("Downloading auxiliary classification file...")
!wget -N https://physionet.org/files/ptb-xl/1.0.1/scp_statements.csv -q

def load_dataset(filename=PTB_XL_META_FILENAME):
    """Loads and preprocesses the PTB-XL metadata."""
    Y = pd.read_csv(filename, index_col='ecg_id')
    # Convert 'scp_codes' from string representation of dictionary to actual dictionary
    Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

    # Load 'super_class' mapping
    agg_df = pd.read_csv('scp_statements.csv', index_col=0)
    agg_df = agg_df[agg_df.diagnostic == 1]

    def aggregate_diagnostic(y_dic):
        """Maps diagnostic codes to superclasses."""
        tmp = []
        for key in y_dic.keys():
            if key in agg_df.index:
                tmp.append(agg_df.loc[key].diagnostic_class)
        return list(set(tmp))

    # Apply the mapping
    Y['diagnosis_super_class'] = Y.scp_codes.apply(aggregate_diagnostic)
    # Filter out rows with no valid superclass
    Y = Y[Y.diagnosis_super_class.apply(lambda x: len(x) > 0)]

    # Prepare labels for Multi-label classification (MultiLabelBinarizer)
    mlb = MultiLabelBinarizer()
    Y_enc = pd.DataFrame(mlb.fit_transform(Y.diagnosis_super_class),
                         columns=mlb.classes_, index=Y.index)

    return Y, Y_enc, mlb.classes_

def load_ecg_signal(filename):
    """Loads a single ECG signal (12 leads) from a .npy file."""
    if SAMPLE_RATE == 100:
        # Assumes the full dataset (records100) is downloaded
        # or that you are loading the full path
        path = DATA_PATH + filename + '.npy'
    else: # SAMPLE_RATE == 500
        path = DATA_PATH + filename.replace('records100','records500') + '.npy'

    # NOTE: The actual PTB-XL signals are stored in a compressed format (e.g., .mat or .npy)
    # The user must ensure the signal files for the 'hr' or 'lr' are accessible.
    # The following line is a placeholder and may require local path adjustments
    # or a script to download the huge signal data.
    try:
        X = np.load(path)
        # Pad or truncate to the fixed length
        if X.shape[1] > N_SAMPLES:
            X = X[:, :N_SAMPLES]
        elif X.shape[1] < N_SAMPLES:
            padding = np.zeros((X.shape[0], N_SAMPLES - X.shape[1]))
            X = np.hstack([X, padding])
        # Reshape to (time_steps, leads) for Keras Conv1D input
        return X.T # (N_LEADS, N_SAMPLES) -> (N_SAMPLES, N_LEADS)
    except FileNotFoundError:
        # A mock signal for a non-downloaded signal to allow the code structure to run
        return np.random.rand(N_SAMPLES, N_LEADS)


# ==============================================================================
# 3. MODEL DEFINITION (1D-CNN / ResNet-like)
# ==============================================================================

def build_resnet_block(input_tensor, n_filters, kernel_size, stride=1):
    """Defines a residual block."""
    x = Conv1D(n_filters, kernel_size, strides=stride, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(n_filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Shortcut connection
    if stride != 1 or input_tensor.shape[-1] != n_filters:
        input_tensor = Conv1D(n_filters, 1, strides=stride, padding='same')(input_tensor)
        input_tensor = BatchNormalization()(input_tensor)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def build_1d_cnn_model(input_shape, n_classes):
    """Defines the 1D-CNN model architecture."""
    input_layer = Input(shape=input_shape)

    # Initial Convolution
    x = Conv1D(64, 15, strides=2, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool1D(pool_size=3, strides=2, padding='same')(x)
    x = Dropout(0.2)(x)

    # Residual Blocks
    x = build_resnet_block(x, 64, 5)
    x = build_resnet_block(x, 64, 5)

    x = build_resnet_block(x, 128, 5, stride=2)
    x = build_resnet_block(x, 128, 5)
    x = Dropout(0.3)(x)

    x = build_resnet_block(x, 256, 5, stride=2)
    x = build_resnet_block(x, 256, 5)
    x = Dropout(0.3)(x)

    # Global Average Pooling and Output
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(n_classes, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy', tf.keras.metrics.F1Score(average='macro')])
    return model

# ==============================================================================
# 4. TRAINING AND EVALUATION
# ==============================================================================
# ==============================================================================
# 4. TRAINING AND EVALUATION (Revised Section)
# ==============================================================================

import os
# ... (Training code 'model.fit(...)' goes here) ...

# Load best weights and evaluate
print("\nEvaluating model on the test set...")

# --- FIX: Check if the best model file exists before loading weights ---
best_weights_file = 'best_model.weights.h5'
if os.path.exists(best_weights_file):
    print(f"Loading weights from {best_weights_file}...")
    try:
        # This will load the best weights saved during training
        model.load_weights(best_weights_file)
    except ValueError as e:
        print(f"Error loading weights: {e}")
        print("\nWARNING: Failed to load best weights. This happens if the checkpoint file was corrupted.")
        print("Proceeding with the weights from the end of the last successful epoch.")
else:
    print(f"WARNING: Checkpoint file '{best_weights_file}' not found.")
    print("This means the training did not complete or did not improve enough to save a checkpoint.")
    print("Proceeding with the weights from the end of the last successful epoch.")

# Now evaluate the model with the best available weights
loss, acc, f1 = model.evaluate(X_test, y_test, verbose=0)
y_pred = (model.predict(X_test) > 0.5).astype(int) # Multi-label threshold
# ... (Rest of the evaluation and printing classification report) ...


# ==============================================================================
# END OF CODE
# ==============================================================================


Evaluating model on the test set...
Loading weights from best_model.weights.h5...
Error loading weights: A total of 32 objects could not be loaded. Example error message for object <Conv1D name=conv1d, built=True>:

Layer 'conv1d' expected 2 variables, but received 0 variables during loading. Expected: ['kernel', 'bias']

List of objects that could not be loaded:
[<Conv1D name=conv1d, built=True>, <BatchNormalization name=batch_normalization, built=True>, <Conv1D name=conv1d_1, built=True>, <BatchNormalization name=batch_normalization_1, built=True>, <Conv1D name=conv1d_2, built=True>, <BatchNormalization name=batch_normalization_2, built=True>, <Conv1D name=conv1d_3, built=True>, <BatchNormalization name=batch_normalization_3, built=True>, <Conv1D name=conv1d_4, built=True>, <BatchNormalization name=batch_normalization_4, built=True>, <Conv1D name=conv1d_5, built=True>, <BatchNormalization name=batch_normalization_5, built=True>, <Conv1D name=conv1d_6, built=True>, <Conv1D name=conv1